<a href="https://colab.research.google.com/github/martinnbl/Dissertation_Project/blob/main/Dissertation_Project_20250802.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Social Media and Marketing Engagement Metrics Pipeline


### Connect to Instagram API

#### Validate API Connection

In [30]:
import requests
import time
from datetime import datetime

# Your existing credentials
user_access_token = "EAAdMs2ZBSSwYBPPncGeEQ9VPKcgBgzuQEPWEEHw43ZA4sqzptgZCSeGZANbdEARzYphNdNItjP2LPnmJ7Vr28u8ngC8Yk3JfYEcvTSZAWNYQUxxypy2ZCGXKNG69Dl9k1ljNfre7avG8MaObVGk57BNgrvzMF2WZA5rbdl0pOZBCZBMxVmBYEw3ZBPHIvtU9wlrZBpEGWebI4efuCSAkHdqYhz7ZCpHqpY4DN5Lji9piQm06ZAchtyNN5"
app_id = "2054658391689990"
app_secret = "1594e932de5d0bd1a3b49979d08e7f37"
app_access_token = f"{app_id}|{app_secret}"

# Track API performance
api_performance = {
    'timestamp': datetime.now().isoformat(),
    'platform': 'Facebook/Meta',
    'operations': []
}

# 1. Token Validation with Performance Tracking
print("🔍 Testing Facebook API Connection...")
start_time = time.time()

try:
    url = "https://graph.facebook.com/debug_token"
    params = {
        "input_token": user_access_token,
        "access_token": app_access_token
    }

    response = requests.get(url, params=params)
    response_time = time.time() - start_time

    # Log results
    operation = {
        'operation': 'Token Validation',
        'status_code': response.status_code,
        'response_time_ms': round(response_time * 1000, 2),
        'success': response.status_code == 200,
        'timestamp': datetime.now().isoformat()
    }

    api_performance['operations'].append(operation)

    if response.status_code == 200:
        token_data = response.json()
        print(f"✅ Token Validation: SUCCESS ({response_time:.3f}s)")
        print(f"   Status Code: {response.status_code}")
        print(f"   Token Valid: {token_data.get('data', {}).get('is_valid', 'Unknown')}")
        print(f"   Permissions: {len(token_data.get('data', {}).get('scopes', []))} scopes")
    else:
        print(f"❌ Token Validation: FAILED ({response_time:.3f}s)")
        print(f"   Status Code: {response.status_code}")
        print(f"   Error: {response.text}")

except Exception as e:
    response_time = time.time() - start_time
    print(f"❌ Connection Error: {e}")
    api_performance['operations'].append({
        'operation': 'Token Validation',
        'status_code': 'ERROR',
        'response_time_ms': round(response_time * 1000, 2),
        'success': False,
        'error': str(e)
    })

# Display Performance Summary
print("\n📊 API Performance Summary:")
for op in api_performance['operations']:
    status = "✅ SUCCESS" if op['success'] else "❌ FAILED"
    print(f"   {op['operation']}: {status} - {op['response_time_ms']}ms")

🔍 Testing Facebook API Connection...
✅ Token Validation: SUCCESS (0.219s)
   Status Code: 200
   Token Valid: True
   Permissions: 8 scopes

📊 API Performance Summary:
   Token Validation: ✅ SUCCESS - 218.6ms


#### Get Data

#### Get only media metrics

In [31]:
import requests
import time
import csv
from datetime import datetime, timezone
import traceback

# === CONFIGURATION ===
ACCESS_TOKEN = user_access_token  # Replace with your token
IG_USER_ID = '17841474204478957'              # Your IG Business Account ID
FILTER_FROM_DATE = datetime.strptime("2024-01-01", "%Y-%m-%d").replace(tzinfo=timezone.utc)

# === PERFORMANCE TRACKING ===
performance_stats = {
    'start_time': time.time(),
    'api_calls': {
        'total': 0,
        'successful': 0,
        'failed': 0,
        'response_times': []
    },
    'data_processing': {
        'items_processed': 0,
        'items_filtered': 0,
        'insights_successful': 0,
        'insights_failed': 0,
        'total_processing_time': 0
    },
    'errors': []
}

def log_api_call(operation, response_time, status_code, success=True):
    """Log API call performance"""
    performance_stats['api_calls']['total'] += 1
    performance_stats['api_calls']['response_times'].append(response_time)

    if success and status_code == 200:
        performance_stats['api_calls']['successful'] += 1
        print(f"✅ {operation}: {response_time:.2f}ms - Status {status_code}")
    else:
        performance_stats['api_calls']['failed'] += 1
        print(f"❌ {operation}: {response_time:.2f}ms - Status {status_code}")

def log_error(component, error, item_id=None):
    """Log errors with context"""
    error_entry = {
        'timestamp': datetime.now().isoformat(),
        'component': component,
        'error_type': type(error).__name__,
        'error_message': str(error),
        'item_id': item_id
    }
    performance_stats['errors'].append(error_entry)
    print(f"🚨 Error in {component}: {error}")

# === FETCH REGULAR MEDIA ===
def get_all_media(ig_user_id, access_token):
    print("📊 Starting media fetch operation...")
    fetch_start = time.time()

    url = f'https://graph.facebook.com/v22.0/{ig_user_id}/media'
    params = {
        'fields': 'username,id,caption,media_type,timestamp,permalink,like_count,comments_count',
        'access_token': access_token
    }

    media_items = []
    page_count = 0

    while url:
        page_start = time.time()

        try:
            res = requests.get(url, params=params if '?' not in url else {})
            response_time = (time.time() - page_start) * 1000

            page_count += 1
            log_api_call(f"Media Fetch Page {page_count}", response_time, res.status_code, res.status_code == 200)

            if res.status_code != 200:
                log_error("Media Fetch", f"HTTP {res.status_code}: {res.text}")
                break

            data = res.json()
            media_items.extend(data.get('data', []))
            url = data.get('paging', {}).get('next', None)

            print(f"📄 Page {page_count}: Retrieved {len(data.get('data', []))} items")
            time.sleep(0.5)

        except Exception as e:
            log_error("Media Fetch", e)
            break

    fetch_duration = time.time() - fetch_start
    print(f"📊 Media fetch completed: {len(media_items)} items in {fetch_duration:.2f}s")

    return media_items

# === FETCH INSIGHTS FOR MEDIA ===
def get_insights_for_media(media_id, media_type, access_token):
    insights_start = time.time()

    url = f'https://graph.facebook.com/v22.0/{media_id}/insights'

    if media_type == 'REEL':
        metrics = 'reach,views,plays,video_views,likes,comments,saved,shares'
    elif media_type == 'VIDEO':
        metrics = 'reach,views,video_views,likes,comments,saved'
    else:
        metrics = 'reach,likes,comments,saved'

    params = {
        'metric': metrics,
        'access_token': access_token
    }

    try:
        res = requests.get(url, params=params)
        response_time = (time.time() - insights_start) * 1000

        log_api_call(f"Insights for {media_type}", response_time, res.status_code, res.status_code == 200)

        metrics_data = {}
        if res.status_code == 200:
            performance_stats['data_processing']['insights_successful'] += 1
            for item in res.json().get('data', []):
                if 'values' in item and item['values']:
                    metrics_data[item['name']] = item['values'][0].get('value', '')
                else:
                    metrics_data[item['name']] = ''
        else:
            performance_stats['data_processing']['insights_failed'] += 1
            log_error("Insights API", f"HTTP {res.status_code}: {res.text}", media_id)

    except Exception as e:
        performance_stats['data_processing']['insights_failed'] += 1
        log_error("Insights API", e, media_id)
        metrics_data = {}

    return metrics_data

# === MAIN FUNCTION ===
def main():
    print("🔍 Fetching posts and stories...")
    print(f"⏰ Process started at: {datetime.now().isoformat()}")

    processing_start = time.time()

    # Fetch all media
    all_media = get_all_media(IG_USER_ID, ACCESS_TOKEN)
    performance_stats['data_processing']['items_processed'] = len(all_media)

    # Filter by date
    filter_start = time.time()
    filtered_items = []

    for item in all_media:
        try:
            media_date = datetime.strptime(item['timestamp'], "%Y-%m-%dT%H:%M:%S%z")
            if media_date >= FILTER_FROM_DATE:
                filtered_items.append(item)
        except Exception as e:
            log_error("Date Filtering", e, item.get('id', 'unknown'))

    filter_duration = time.time() - filter_start
    performance_stats['data_processing']['items_filtered'] = len(filtered_items)

    print(f"✅ {len(filtered_items)} media items found after {FILTER_FROM_DATE.date()}")
    print(f"⏱️ Filtering completed in {filter_duration:.2f}s")

    # Process insights for each item
    insights_start = time.time()
    results = []

    for i, item in enumerate(filtered_items, 1):
        print(f"🔄 Processing item {i}/{len(filtered_items)}: {item['media_type']}")

        media_type = item['media_type']
        insights = get_insights_for_media(item['id'], media_type, ACCESS_TOKEN)

        results.append({
            'username': item.get('username', ''),
            'permalink': item.get('permalink', ''),
            'id': item['id'],
            'caption': item.get('caption', ''),
            'timestamp': item['timestamp'],
            'media_type': media_type,
            'like_count': item.get('like_count', '') if media_type != 'STORY' else '',
            'reach': insights.get('reach', ''),
            'video_views': insights.get('views', '') if media_type in ['REEL', 'VIDEO'] else '',
            'plays': insights.get('plays', '') if media_type in ['REEL'] else '',
            'Platform': 'Instagram',
        })
        time.sleep(0.5)

    insights_duration = time.time() - insights_start

    # Save results
    export_start = time.time()
    if results:
        with open('instagram_media_metrics.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=results[0].keys())
            writer.writeheader()
            writer.writerows(results)
        export_duration = time.time() - export_start
        print(f"📁 Data saved to instagram_media_metrics.csv in {export_duration:.2f}s")
    else:
        print("⚠️ No media to save.")

    # Calculate final stats
    total_duration = time.time() - processing_start
    performance_stats['data_processing']['total_processing_time'] = total_duration

    # === PERFORMANCE SUMMARY ===
    print("\n" + "="*60)
    print("📊 PERFORMANCE SUMMARY")
    print("="*60)

    print(f"⏱️ Total Execution Time: {total_duration:.2f} seconds")
    print(f"🌐 API Calls Made: {performance_stats['api_calls']['total']}")
    print(f"✅ Successful API Calls: {performance_stats['api_calls']['successful']}")
    print(f"❌ Failed API Calls: {performance_stats['api_calls']['failed']}")

    if performance_stats['api_calls']['response_times']:
        avg_response = sum(performance_stats['api_calls']['response_times']) / len(performance_stats['api_calls']['response_times'])
        max_response = max(performance_stats['api_calls']['response_times'])
        min_response = min(performance_stats['api_calls']['response_times'])
        print(f"📈 Average API Response Time: {avg_response:.2f}ms")
        print(f"📈 Max API Response Time: {max_response:.2f}ms")
        print(f"📈 Min API Response Time: {min_response:.2f}ms")

    print(f"📊 Items Processed: {performance_stats['data_processing']['items_processed']}")
    print(f"🔍 Items After Filtering: {performance_stats['data_processing']['items_filtered']}")
    print(f"✅ Successful Insights Calls: {performance_stats['data_processing']['insights_successful']}")
    print(f"❌ Failed Insights Calls: {performance_stats['data_processing']['insights_failed']}")

    # Success rates
    if performance_stats['api_calls']['total'] > 0:
        api_success_rate = (performance_stats['api_calls']['successful'] / performance_stats['api_calls']['total']) * 100
        print(f"📊 Overall API Success Rate: {api_success_rate:.1f}%")

    insights_total = performance_stats['data_processing']['insights_successful'] + performance_stats['data_processing']['insights_failed']
    if insights_total > 0:
        insights_success_rate = (performance_stats['data_processing']['insights_successful'] / insights_total) * 100
        print(f"📊 Insights API Success Rate: {insights_success_rate:.1f}%")

    # Error summary
    if performance_stats['errors']:
        print(f"\n🚨 Errors Encountered: {len(performance_stats['errors'])}")
        for error in performance_stats['errors']:
            print(f"   - {error['component']}: {error['error_type']}")
    else:
        print("\n✅ No errors encountered!")

    print("="*60)

    # Return stats for further analysis
    return performance_stats, results

if __name__ == "__main__":
    stats, data = main()

🔍 Fetching posts and stories...
⏰ Process started at: 2025-08-02T14:41:58.023750
📊 Starting media fetch operation...
✅ Media Fetch Page 1: 245.27ms - Status 200
📄 Page 1: Retrieved 4 items
📊 Media fetch completed: 4 items in 0.75s
✅ 4 media items found after 2024-01-01
⏱️ Filtering completed in 0.00s
🔄 Processing item 1/4: VIDEO
❌ Insights for VIDEO: 169.25ms - Status 400
🚨 Error in Insights API: HTTP 400: {"error":{"message":"(#100) The Media Insights API does not support the video_views metric for this media product type.","type":"OAuthException","code":100,"fbtrace_id":"Ar3l718E_evywPkKX6qQ7Et"}}
🔄 Processing item 2/4: IMAGE
✅ Insights for IMAGE: 285.95ms - Status 200
🔄 Processing item 3/4: IMAGE
✅ Insights for IMAGE: 299.05ms - Status 200
🔄 Processing item 4/4: IMAGE
✅ Insights for IMAGE: 260.87ms - Status 200
📁 Data saved to instagram_media_metrics.csv in 0.00s

📊 PERFORMANCE SUMMARY
⏱️ Total Execution Time: 3.77 seconds
🌐 API Calls Made: 5
✅ Successful API Calls: 4
❌ Failed API C

In [32]:
import pandas as pd

In [33]:
df_2= pd.read_csv('instagram_media_metrics.csv')
df_2


,username,permalink,id,caption,timestamp,media_type,like_count,reach,video_views,plays,Platform
0,mnblanalytics,https://www.instagram.com/reel/DKWmrCONQqs/,18080236366793735,Testreel,2025-06-01T09:30:07+0000,VIDEO,5,NaN,NaN,NaN,Instagram
1,mnblanalytics,https://www.instagram.com/p/DJ4VPz6tPPb/,18080278336761002,#Iniciative1,2025-05-20T15:20:22+0000,IMAGE,5,32.0,NaN,NaN,Instagram
2,mnblanalytics,https://www.instagram.com/p/DJb6pZWtXKt/,17904426585155969,Test 2,2025-05-09T14:29:12+0000,IMAGE,6,40.0,NaN,NaN,Instagram
3,mnblanalytics,https://www.instagram.com/p/DJb6PloNTog/,18079149412810358,Test 1,2025-05-09T14:25:41+0000,IMAGE,7,38.0,NaN,NaN,Instagram


### Get only Stories Data

In [34]:
import requests
import time
import pandas as pd
from datetime import datetime
import traceback

ACCESS_TOKEN = user_access_token
IG_USER_ID = '17841474204478957'

# === PERFORMANCE TRACKING ===
stories_performance_stats = {
    'start_time': time.time(),
    'api_calls': {
        'total': 0,
        'successful': 0,
        'failed': 0,
        'response_times': []
    },
    'stories_processing': {
        'stories_found': 0,
        'stories_processed': 0,
        'insights_successful': 0,
        'insights_failed': 0,
        'insights_insufficient_data': 0,
        'total_processing_time': 0
    },
    'errors': [],
    'data_quality': {
        'complete_stories': 0,
        'stories_with_insights': 0,
        'stories_without_insights': 0
    }
}

def log_stories_api_call(operation, response_time, status_code, success=True, data_count=0):
    """Log API call performance for stories"""
    stories_performance_stats['api_calls']['total'] += 1
    stories_performance_stats['api_calls']['response_times'].append(response_time)

    if success and status_code == 200:
        stories_performance_stats['api_calls']['successful'] += 1
        print(f"✅ {operation}: {response_time:.2f}ms - Status {status_code} - {data_count} items")
    else:
        stories_performance_stats['api_calls']['failed'] += 1
        print(f"❌ {operation}: {response_time:.2f}ms - Status {status_code}")

def log_stories_error(component, error, story_id=None):
    """Log errors with context for stories"""
    error_entry = {
        'timestamp': datetime.now().isoformat(),
        'component': component,
        'error_type': type(error).__name__,
        'error_message': str(error),
        'story_id': story_id
    }
    stories_performance_stats['errors'].append(error_entry)
    print(f"🚨 Stories Error in {component}: {error}")

def get_stories(ig_user_id, access_token):
    print("📊 Starting stories fetch operation...")
    fetch_start = time.time()

    url = f'https://graph.facebook.com/v19.0/{ig_user_id}/stories'
    params = {
        'fields': 'username,id,media_type,media_url,timestamp,permalink,caption',
        'access_token': access_token
    }

    try:
        res = requests.get(url, params=params)
        response_time = (time.time() - fetch_start) * 1000

        stories_data = []
        if res.status_code == 200:
            stories_data = res.json().get('data', [])
            stories_performance_stats['stories_processing']['stories_found'] = len(stories_data)
            log_stories_api_call("Stories Fetch", response_time, res.status_code, True, len(stories_data))
        else:
            log_stories_api_call("Stories Fetch", response_time, res.status_code, False)
            log_stories_error("Stories Fetch", f"HTTP {res.status_code}: {res.text}")

        return stories_data

    except Exception as e:
        response_time = (time.time() - fetch_start) * 1000
        log_stories_error("Stories Fetch", e)
        return []

def get_story_insights(media_id, access_token):
    insights_start = time.time()

    # Allowed metrics for story insights in latest API versions
    metrics = "reach,replies,total_interactions,views"
    url = f'https://graph.facebook.com/v22.0/{media_id}/insights'
    params = {
        'metric': metrics,
        'access_token': access_token
    }

    try:
        res = requests.get(url, params=params)
        response_time = (time.time() - insights_start) * 1000

        insights = {}
        if res.status_code == 200:
            stories_performance_stats['stories_processing']['insights_successful'] += 1
            stories_performance_stats['data_quality']['stories_with_insights'] += 1

            data = res.json().get('data', [])
            for item in data:
                if 'values' in item and item['values']:
                    insights[item['name']] = item['values'][0]['value']
                else:
                    insights[item['name']] = 0

            log_stories_api_call(f"Story Insights", response_time, res.status_code, True, len(insights))

        else:
            stories_performance_stats['stories_processing']['insights_failed'] += 1
            stories_performance_stats['data_quality']['stories_without_insights'] += 1

            # Check if it's insufficient data error
            if "Not enough viewers" in res.text or "insufficient data" in res.text.lower():
                stories_performance_stats['stories_processing']['insights_insufficient_data'] += 1
                print(f"ℹ️ Insufficient data for story {media_id} insights (normal for low-view stories)")
            else:
                log_stories_error("Story Insights", f"HTTP {res.status_code}: {res.text}", media_id)

            log_stories_api_call(f"Story Insights", response_time, res.status_code, False)

    except Exception as e:
        stories_performance_stats['stories_processing']['insights_failed'] += 1
        stories_performance_stats['data_quality']['stories_without_insights'] += 1
        log_stories_error("Story Insights", e, media_id)
        insights = {}

    return insights

def main():
    print("🔍 Fetching Instagram Stories...")
    print(f"⏰ Stories process started at: {datetime.now().isoformat()}")

    processing_start = time.time()

    # Fetch stories
    stories = get_stories(IG_USER_ID, ACCESS_TOKEN)

    if not stories:
        print("⚠️ No stories found.")
        # Still generate performance report
        generate_stories_performance_report(processing_start, [])
        return []

    print(f"📊 Found {len(stories)} active stories")

    # Process each story
    story_data = []
    stories_performance_stats['stories_processing']['stories_processed'] = len(stories)

    for i, story in enumerate(stories, 1):
        print(f"\n🔄 Processing story {i}/{len(stories)}")
        print(f"   Story ID: {story.get('id')}")
        print(f"   Type: {story.get('media_type')}")
        print(f"   Timestamp: {story.get('timestamp')}")
        print(f"   Caption: {story.get('caption', 'No caption')}")

        story_processing_start = time.time()
        insights = get_story_insights(story['id'], ACCESS_TOKEN)
        story_processing_time = time.time() - story_processing_start

        # Build story data record
        story_record = {
            'username': story.get('username', ''),
            'id': story.get('id'),
            'media_type': story.get('media_type'),
            'timestamp': story.get('timestamp'),
            'permalink': story.get('permalink'),
            'caption': story.get('caption', ''),
            'reach': insights.get('reach', 0),
            'replies': insights.get('replies', 0),
            'total_interactions': insights.get('total_interactions', 0),
            'views': insights.get('views', 0),
            'Platform': 'Instagram',
            'processing_time_ms': round(story_processing_time * 1000, 2)
        }

        story_data.append(story_record)

        # Check data completeness
        if any(insights.values()):
            stories_performance_stats['data_quality']['complete_stories'] += 1

        # Display insights
        if insights:
            for k, v in insights.items():
                print(f"   {k.capitalize()}: {v}")
        else:
            print("   ℹ️ No insights available (insufficient data)")

        print('-' * 50)
        time.sleep(0.5)  # Be kind to the API rate limits

    # Convert to DataFrame
    df_stories = pd.DataFrame(story_data)

    # Save to CSV
    export_start = time.time()
    try:
        df_stories.to_csv('instagram_stories_detailed.csv', index=False)
        export_duration = time.time() - export_start
        print(f"📁 Stories data saved to instagram_stories_detailed.csv in {export_duration:.2f}s")
    except Exception as e:
        log_stories_error("CSV Export", e)

    # Generate performance report
    generate_stories_performance_report(processing_start, story_data)

    return story_data

def generate_stories_performance_report(processing_start, story_data):
    """Generate comprehensive performance report"""
    total_duration = time.time() - processing_start
    stories_performance_stats['stories_processing']['total_processing_time'] = total_duration

    print("\n" + "="*60)
    print("📊 INSTAGRAM STORIES PERFORMANCE SUMMARY")
    print("="*60)

    # Timing metrics
    print(f"⏱️ Total Execution Time: {total_duration:.2f} seconds")

    # API performance
    print(f"🌐 Total API Calls: {stories_performance_stats['api_calls']['total']}")
    print(f"✅ Successful API Calls: {stories_performance_stats['api_calls']['successful']}")
    print(f"❌ Failed API Calls: {stories_performance_stats['api_calls']['failed']}")

    if stories_performance_stats['api_calls']['response_times']:
        response_times = stories_performance_stats['api_calls']['response_times']
        avg_response = sum(response_times) / len(response_times)
        max_response = max(response_times)
        min_response = min(response_times)
        print(f"📈 Average API Response Time: {avg_response:.2f}ms")
        print(f"📈 Max API Response Time: {max_response:.2f}ms")
        print(f"📈 Min API Response Time: {min_response:.2f}ms")

    # Stories processing metrics
    print(f"📱 Stories Found: {stories_performance_stats['stories_processing']['stories_found']}")
    print(f"🔄 Stories Processed: {stories_performance_stats['stories_processing']['stories_processed']}")
    print(f"✅ Insights Retrieved: {stories_performance_stats['stories_processing']['insights_successful']}")
    print(f"❌ Insights Failed: {stories_performance_stats['stories_processing']['insights_failed']}")
    print(f"ℹ️ Insufficient Data Cases: {stories_performance_stats['stories_processing']['insights_insufficient_data']}")

    # Success rates
    if stories_performance_stats['api_calls']['total'] > 0:
        api_success_rate = (stories_performance_stats['api_calls']['successful'] / stories_performance_stats['api_calls']['total']) * 100
        print(f"📊 Overall API Success Rate: {api_success_rate:.1f}%")

    insights_total = stories_performance_stats['stories_processing']['insights_successful'] + stories_performance_stats['stories_processing']['insights_failed']
    if insights_total > 0:
        insights_success_rate = (stories_performance_stats['stories_processing']['insights_successful'] / insights_total) * 100
        print(f"📊 Insights Success Rate: {insights_success_rate:.1f}%")

    # Data quality metrics
    print(f"\n📊 DATA QUALITY METRICS:")
    print(f"   Complete Stories (with insights): {stories_performance_stats['data_quality']['complete_stories']}")
    print(f"   Stories with Insights: {stories_performance_stats['data_quality']['stories_with_insights']}")
    print(f"   Stories without Insights: {stories_performance_stats['data_quality']['stories_without_insights']}")

    if story_data:
        data_completeness = (stories_performance_stats['data_quality']['complete_stories'] / len(story_data)) * 100
        print(f"   Data Completeness Rate: {data_completeness:.1f}%")

    # Error summary
    if stories_performance_stats['errors']:
        print(f"\n🚨 Errors Encountered: {len(stories_performance_stats['errors'])}")
        error_types = {}
        for error in stories_performance_stats['errors']:
            error_type = f"{error['component']}: {error['error_type']}"
            error_types[error_type] = error_types.get(error_type, 0) + 1

        for error_type, count in error_types.items():
            print(f"   - {error_type}: {count} occurrence(s)")
    else:
        print("\n✅ No errors encountered!")

    # Performance insights
    print(f"\n💡 PERFORMANCE INSIGHTS:")
    if stories_performance_stats['stories_processing']['insights_insufficient_data'] > 0:
        print(f"   - {stories_performance_stats['stories_processing']['insights_insufficient_data']} stories had insufficient data for insights (normal)")

    if stories_performance_stats['api_calls']['response_times']:
        avg_time = sum(stories_performance_stats['api_calls']['response_times']) / len(stories_performance_stats['api_calls']['response_times'])
        if avg_time > 1000:
            print(f"   - API response times averaging {avg_time:.0f}ms suggest potential network/server delays")
        else:
            print(f"   - API response times averaging {avg_time:.0f}ms indicate good performance")

    print("="*60)

    return stories_performance_stats

if __name__ == "__main__":
    story_results = main()

🔍 Fetching Instagram Stories...
⏰ Stories process started at: 2025-08-02T14:42:16.866177
📊 Starting stories fetch operation...
✅ Stories Fetch: 386.31ms - Status 200 - 2 items
📊 Found 2 active stories

🔄 Processing story 1/2
   Story ID: 17929674428964127
   Type: IMAGE
   Timestamp: 2025-08-02T13:54:17+0000
   Caption: TEST 3
✅ Story Insights: 426.05ms - Status 200 - 4 items
   Reach: 11
   Replies: 0
   Total_interactions: 0
   Views: 12
--------------------------------------------------

🔄 Processing story 2/2
   Story ID: 18016504853574187
   Type: IMAGE
   Timestamp: 2025-08-02T13:53:44+0000
   Caption: Test
✅ Story Insights: 468.59ms - Status 200 - 4 items
   Reach: 12
   Replies: 0
   Total_interactions: 0
   Views: 13
--------------------------------------------------
📁 Stories data saved to instagram_stories_detailed.csv in 0.00s

📊 INSTAGRAM STORIES PERFORMANCE SUMMARY
⏱️ Total Execution Time: 2.29 seconds
🌐 Total API Calls: 3
✅ Successful API Calls: 3
❌ Failed API Calls: 0
📈

In [35]:
import requests
import time
import pandas as pd

ACCESS_TOKEN = user_access_token
IG_USER_ID = '17841474204478957'

def get_stories(ig_user_id, access_token):
    url = f'https://graph.facebook.com/v19.0/{ig_user_id}/stories'
    params = {
        'fields': 'username,id,media_type,media_url,timestamp,permalink,caption',
        'access_token': access_token
    }
    res = requests.get(url, params=params)
    if res.status_code == 200:
        return res.json().get('data', [])
    else:
        print(f"Error fetching stories: {res.status_code} {res.text}")
        return []

def get_story_insights(media_id, access_token):
    metrics = "reach,replies,total_interactions,views"
    url = f'https://graph.facebook.com/v22.0/{media_id}/insights'
    params = {
        'metric': metrics,
        'access_token': access_token
    }
    res = requests.get(url, params=params)
    insights = {}
    if res.status_code == 200:
        for item in res.json().get('data', []):
            insights[item['name']] = item['values'][0]['value']
    else:
        print(f"Error getting insights for story {media_id}: {res.status_code} {res.text}")
    return insights

def main():
    stories = get_stories(IG_USER_ID, ACCESS_TOKEN)
    if not stories:
        print("No stories found.")
        return

    story_data = []

    for story in stories:
        insights = get_story_insights(story['id'], ACCESS_TOKEN)
        story_row = {
           'username': story.get('username'),
            'id': story.get('id'),
            'media_type': story.get('media_type'),
            #'media_url': story.get('media_url'),
            'timestamp': story.get('timestamp'),
            'permalink': story.get('permalink'),
            'caption': story.get('caption', ''),
            'reach': insights.get('reach', 0),
            'replies': insights.get('replies', 0),
            'total_interactions': insights.get('total_interactions', 0),
            'views': insights.get('views', 0),
            'Platform':'Instagram',
        }
        story_data.append(story_row)
        time.sleep(0.5)

    df_stories = pd.DataFrame(story_data)
    print(df_stories)

    # Optional: Save to CSV or Excel
    df_stories.to_csv('instagram_stories.csv', index=False)

    return df_stories

if __name__ == "__main__":
    df = main()


        username                 id media_type                 timestamp  \
0  mnblanalytics  17929674428964127      IMAGE  2025-08-02T13:54:17+0000   
1  mnblanalytics  18016504853574187      IMAGE  2025-08-02T13:53:44+0000   

                                           permalink caption  reach  replies  \
0  https://instagram.com/stories/mnblanalytics/36...  TEST 3     11        0   
1  https://instagram.com/stories/mnblanalytics/36...    Test     12        0   

   total_interactions  views   Platform  
0                   0     12  Instagram  
1                   0     13  Instagram  


In [36]:
df_3= pd.read_csv('instagram_stories.csv')
df_3

,username,id,media_type,timestamp,permalink,caption,reach,replies,total_interactions,views,Platform
0,mnblanalytics,17929674428964127,IMAGE,2025-08-02T13:54:17+0000,https://instagram.com/stories/mnblanalytics/36...,TEST 3,11,0,0,12,Instagram
1,mnblanalytics,18016504853574187,IMAGE,2025-08-02T13:53:44+0000,https://instagram.com/stories/mnblanalytics/36...,Test,12,0,0,13,Instagram


### Concatenate Ig Media & Ig Stories Data

In [37]:
import pandas as pd
import time
from datetime import datetime
import os

# === DATA CONCATENATION PERFORMANCE TRACKING ===
concatenation_stats = {
    'start_time': time.time(),
    'file_operations': {
        'files_loaded': 0,
        'load_times': {},
        'file_sizes': {},
        'load_errors': []
    },
    'data_processing': {
        'total_records_media': 0,
        'total_records_stories': 0,
        'total_records_combined': 0,
        'columns_media': 0,
        'columns_stories': 0,
        'columns_combined': 0,
        'missing_values_created': 0,
        'processing_time': 0
    },
    'data_quality': {
        'duplicate_records': 0,
        'data_completeness': {},
        'column_overlap': [],
        'unique_columns': []
    }
}

def log_file_operation(operation, file_name, duration, success=True, error=None):
    """Log file loading operations"""
    if success:
        concatenation_stats['file_operations']['files_loaded'] += 1
        concatenation_stats['file_operations']['load_times'][file_name] = duration
        print(f"✅ {operation}: {file_name} loaded in {duration:.3f}s")
    else:
        concatenation_stats['file_operations']['load_errors'].append({
            'file': file_name,
            'error': str(error),
            'timestamp': datetime.now().isoformat()
        })
        print(f"❌ {operation}: Failed to load {file_name} - {error}")

def analyze_dataframe(df, df_name):
    """Analyze dataframe properties"""
    analysis = {
        'name': df_name,
        'rows': len(df),
        'columns': len(df.columns),
        'memory_usage_mb': df.memory_usage(deep=True).sum() / 1024 / 1024,
        'null_values': df.isnull().sum().sum(),
        'duplicate_rows': df.duplicated().sum(),
        'column_list': df.columns.tolist(),
        'dtypes': df.dtypes.to_dict()
    }

    print(f"📊 {df_name} Analysis:")
    print(f"   Rows: {analysis['rows']:,}")
    print(f"   Columns: {analysis['columns']}")
    print(f"   Memory Usage: {analysis['memory_usage_mb']:.2f} MB")
    print(f"   Null Values: {analysis['null_values']:,}")
    print(f"   Duplicate Rows: {analysis['duplicate_rows']}")
    print(f"   Columns: {analysis['column_list']}")

    return analysis

def main():
    print("🔄 Starting Instagram Data Concatenation Process")
    print(f"⏰ Process started at: {datetime.now().isoformat()}")
    print("="*60)

    # === STEP 1: LOAD DATAFRAMES ===
    print("\n📂 STEP 1: Loading CSV Files")

    # Load media data
    media_load_start = time.time()
    try:
        if os.path.exists('instagram_media_metrics.csv'):
            df_media = pd.read_csv('instagram_media_metrics.csv')
            media_load_time = time.time() - media_load_start
            log_file_operation("Media Load", "instagram_media_metrics.csv", media_load_time, True)

            # Get file size
            file_size_mb = os.path.getsize('instagram_media_metrics.csv') / 1024 / 1024
            concatenation_stats['file_operations']['file_sizes']['media'] = file_size_mb

        else:
            raise FileNotFoundError("instagram_media_metrics.csv not found")

    except Exception as e:
        media_load_time = time.time() - media_load_start
        log_file_operation("Media Load", "instagram_media_metrics.csv", media_load_time, False, e)
        return None

    # Load stories data
    stories_load_start = time.time()
    try:
        if os.path.exists('instagram_stories.csv'):
            df_stories = pd.read_csv('instagram_stories.csv')
            stories_load_time = time.time() - stories_load_start
            log_file_operation("Stories Load", "instagram_stories.csv", stories_load_time, True)

            # Get file size
            file_size_mb = os.path.getsize('instagram_stories.csv') / 1024 / 1024
            concatenation_stats['file_operations']['file_sizes']['stories'] = file_size_mb

        else:
            raise FileNotFoundError("instagram_stories.csv not found")

    except Exception as e:
        stories_load_time = time.time() - stories_load_start
        log_file_operation("Stories Load", "instagram_stories.csv", stories_load_time, False, e)
        return None

    # === STEP 2: ANALYZE SOURCE DATA ===
    print("\n📊 STEP 2: Analyzing Source Data")

    media_analysis = analyze_dataframe(df_media, "Media DataFrame")
    stories_analysis = analyze_dataframe(df_stories, "Stories DataFrame")

    # Store in stats
    concatenation_stats['data_processing']['total_records_media'] = len(df_media)
    concatenation_stats['data_processing']['total_records_stories'] = len(df_stories)
    concatenation_stats['data_processing']['columns_media'] = len(df_media.columns)
    concatenation_stats['data_processing']['columns_stories'] = len(df_stories.columns)

    # Analyze column overlap
    media_cols = set(df_media.columns)
    stories_cols = set(df_stories.columns)
    common_cols = media_cols.intersection(stories_cols)
    unique_media_cols = media_cols - stories_cols
    unique_stories_cols = stories_cols - media_cols

    concatenation_stats['data_quality']['column_overlap'] = list(common_cols)
    concatenation_stats['data_quality']['unique_columns'] = {
        'media_only': list(unique_media_cols),
        'stories_only': list(unique_stories_cols)
    }

    print(f"\n🔍 Column Analysis:")
    print(f"   Common columns: {len(common_cols)} - {list(common_cols)}")
    print(f"   Media-only columns: {len(unique_media_cols)} - {list(unique_media_cols)}")
    print(f"   Stories-only columns: {len(unique_stories_cols)} - {list(unique_stories_cols)}")

    # === STEP 3: ADD CATEGORY COLUMNS ===
    print("\n🏷️ STEP 3: Adding Category Identifiers")

    category_start = time.time()
    df_media['media_category'] = 'media'
    df_stories['media_category'] = 'story'
    category_time = time.time() - category_start

    print(f"✅ Category columns added in {category_time:.4f}s")

    # === STEP 4: CONCATENATE DATAFRAMES ===
    print("\n🔗 STEP 4: Concatenating DataFrames")

    concat_start = time.time()
    try:
        # Concatenate with detailed tracking
        combined_df = pd.concat([df_media, df_stories], ignore_index=True, sort=False)
        concat_time = time.time() - concat_start
        concatenation_stats['data_processing']['processing_time'] = concat_time

        print(f"✅ Concatenation completed in {concat_time:.4f}s")

    except Exception as e:
        concat_time = time.time() - concat_start
        print(f"❌ Concatenation failed after {concat_time:.4f}s: {e}")
        return None

    # === STEP 5: ANALYZE COMBINED DATA ===
    print("\n📈 STEP 5: Analyzing Combined Dataset")

    combined_analysis = analyze_dataframe(combined_df, "Combined DataFrame")

    # Store combined stats
    concatenation_stats['data_processing']['total_records_combined'] = len(combined_df)
    concatenation_stats['data_processing']['columns_combined'] = len(combined_df.columns)
    concatenation_stats['data_processing']['missing_values_created'] = combined_df.isnull().sum().sum()

    # Calculate data completeness for each column
    completeness = {}
    for col in combined_df.columns:
        non_null_count = combined_df[col].notna().sum()
        completeness[col] = (non_null_count / len(combined_df)) * 100

    concatenation_stats['data_quality']['data_completeness'] = completeness

    # Check for duplicates
    concatenation_stats['data_quality']['duplicate_records'] = combined_df.duplicated().sum()

    print(f"\n📋 Data Completeness by Column:")
    for col, comp_pct in sorted(completeness.items(), key=lambda x: x[1], reverse=True):
        print(f"   {col}: {comp_pct:.1f}%")

    # === STEP 6: DISPLAY RESULTS ===
    print("\n📄 STEP 6: Sample Data Preview")
    print("\nFirst 5 rows of combined dataframe:")
    print(combined_df.head())

    print(f"\nDataFrame Info:")
    print(combined_df.info())

    # === STEP 7: SAVE COMBINED DATA ===
    print("\n💾 STEP 7: Saving Combined Dataset")

    save_start = time.time()
    try:
        output_filename = f'instagram_combined_data_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
        combined_df.to_csv(output_filename, index=False)
        save_time = time.time() - save_start

        output_size_mb = os.path.getsize(output_filename) / 1024 / 1024
        print(f"✅ Combined data saved to {output_filename}")
        print(f"   Save time: {save_time:.3f}s")
        print(f"   File size: {output_size_mb:.2f} MB")

    except Exception as e:
        save_time = time.time() - save_start
        print(f"❌ Failed to save combined data after {save_time:.3f}s: {e}")

    # === GENERATE PERFORMANCE REPORT ===
    generate_concatenation_report()

    return combined_df

def generate_concatenation_report():
    """Generate comprehensive performance report for data concatenation"""
    total_duration = time.time() - concatenation_stats['start_time']

    print("\n" + "="*60)
    print("📊 DATA CONCATENATION PERFORMANCE REPORT")
    print("="*60)

    # Overall performance
    print(f"⏱️ Total Processing Time: {total_duration:.3f} seconds")
    print(f"📁 Files Loaded: {concatenation_stats['file_operations']['files_loaded']}")

    # File loading performance
    print(f"\n📂 FILE LOADING PERFORMANCE:")
    for file_name, load_time in concatenation_stats['file_operations']['load_times'].items():
        file_type = 'Media' if 'media' in file_name else 'Stories'
        size_mb = concatenation_stats['file_operations']['file_sizes'].get(file_type.lower(), 0)
        print(f"   {file_type}: {load_time:.3f}s ({size_mb:.2f} MB)")

    # Data processing metrics
    print(f"\n📊 DATA PROCESSING METRICS:")
    print(f"   Media Records: {concatenation_stats['data_processing']['total_records_media']:,}")
    print(f"   Stories Records: {concatenation_stats['data_processing']['total_records_stories']:,}")
    print(f"   Combined Records: {concatenation_stats['data_processing']['total_records_combined']:,}")
    print(f"   Processing Time: {concatenation_stats['data_processing']['processing_time']:.4f}s")

    # Column analysis
    print(f"\n🏛️ COLUMN STRUCTURE:")
    print(f"   Media Columns: {concatenation_stats['data_processing']['columns_media']}")
    print(f"   Stories Columns: {concatenation_stats['data_processing']['columns_stories']}")
    print(f"   Combined Columns: {concatenation_stats['data_processing']['columns_combined']}")
    print(f"   Common Columns: {len(concatenation_stats['data_quality']['column_overlap'])}")

    # Data quality metrics
    print(f"\n🎯 DATA QUALITY METRICS:")
    print(f"   Missing Values Created: {concatenation_stats['data_processing']['missing_values_created']:,}")
    print(f"   Duplicate Records: {concatenation_stats['data_quality']['duplicate_records']}")

    # Performance insights
    if concatenation_stats['data_processing']['total_records_combined'] > 0:
        processing_rate = concatenation_stats['data_processing']['total_records_combined'] / concatenation_stats['data_processing']['processing_time']
        print(f"   Processing Rate: {processing_rate:,.0f} records/second")

    # Column completeness summary
    if concatenation_stats['data_quality']['data_completeness']:
        avg_completeness = sum(concatenation_stats['data_quality']['data_completeness'].values()) / len(concatenation_stats['data_quality']['data_completeness'])
        print(f"   Average Column Completeness: {avg_completeness:.1f}%")

    # Error summary
    if concatenation_stats['file_operations']['load_errors']:
        print(f"\n🚨 ERRORS ENCOUNTERED:")
        for error in concatenation_stats['file_operations']['load_errors']:
            print(f"   {error['file']}: {error['error']}")
    else:
        print(f"\n✅ No errors encountered during processing!")

    print("="*60)

    return concatenation_stats

if __name__ == "__main__":
    # Execute the concatenation process
    result_df = main()

    if result_df is not None:
        print(f"\n🎉 Concatenation process completed successfully!")
        print(f"📊 Final dataset shape: {result_df.shape}")
    else:
        print(f"\n❌ Concatenation process failed!")

🔄 Starting Instagram Data Concatenation Process
⏰ Process started at: 2025-08-02T14:42:46.929122

📂 STEP 1: Loading CSV Files
✅ Media Load: instagram_media_metrics.csv loaded in 0.003s
✅ Stories Load: instagram_stories.csv loaded in 0.001s

📊 STEP 2: Analyzing Source Data
📊 Media DataFrame Analysis:
   Rows: 4
   Columns: 11
   Memory Usage: 0.00 MB
   Null Values: 9
   Duplicate Rows: 0
   Columns: ['username', 'permalink', 'id', 'caption', 'timestamp', 'media_type', 'like_count', 'reach', 'video_views', 'plays', 'Platform']
📊 Stories DataFrame Analysis:
   Rows: 2
   Columns: 11
   Memory Usage: 0.00 MB
   Null Values: 0
   Duplicate Rows: 0
   Columns: ['username', 'id', 'media_type', 'timestamp', 'permalink', 'caption', 'reach', 'replies', 'total_interactions', 'views', 'Platform']

🔍 Column Analysis:
   Common columns: 8 - ['username', 'reach', 'timestamp', 'caption', 'permalink', 'Platform', 'id', 'media_type']
   Media-only columns: 3 - ['video_views', 'plays', 'like_count']
   

In [38]:
combined_df

,username,permalink,id,caption,timestamp,media_type,like_count,reach,video_views,plays,Platform,media_category,replies,total_interactions,views
0,mnblanalytics,https://www.instagram.com/reel/DKWmrCONQqs/,18080236366793735,Testreel,2025-06-01T09:30:07+0000,VIDEO,5.0,NaN,NaN,NaN,Instagram,media,NaN,NaN,NaN
1,mnblanalytics,https://www.instagram.com/p/DJ4VPz6tPPb/,18080278336761002,#Iniciative1,2025-05-20T15:20:22+0000,IMAGE,5.0,32.0,NaN,NaN,Instagram,media,NaN,NaN,NaN
2,mnblanalytics,https://www.instagram.com/p/DJb6pZWtXKt/,17904426585155969,Test 2,2025-05-09T14:29:12+0000,IMAGE,6.0,40.0,NaN,NaN,Instagram,media,NaN,NaN,NaN
3,mnblanalytics,https://www.instagram.com/p/DJb6PloNTog/,18079149412810358,Test 1,2025-05-09T14:25:41+0000,IMAGE,7.0,38.0,NaN,NaN,Instagram,media,NaN,NaN,NaN
4,mnblanalytics,https://instagram.com/stories/mnblanalytics/36...,17929674428964127,TEST 3,2025-08-02T13:54:17+0000,IMAGE,NaN,0.0,NaN,NaN,Instagram,story,0.0,0.0,0.0
5,mnblanalytics,https://instagram.com/stories/mnblanalytics/36...,18016504853574187,Test,2025-08-02T13:53:44+0000,IMAGE,NaN,0.0,NaN,NaN,Instagram,story,0.0,0.0,0.0


## Generate DDBB in Big Query

In [19]:
!pip install --upgrade google-cloud-bigquery pandas-gbq

### Authenticate

In [20]:
from google.colab import auth
auth.authenticate_user()

### Upload IG Data Frame to Big Query

In [41]:
from google.cloud import bigquery
from pandas_gbq import to_gbq
import time

project_id = "proof-of-brand"
dataset_id = "social_media_metrics"
table_id = "instagram_data"

# Start timing
start_time = time.time()

print(f"Starting upload of {len(combined_df)} rows to BigQuery...")
print(f"Target: {project_id}.{dataset_id}.{table_id}")

to_gbq(
    combined_df,
    f"{dataset_id}.{table_id}",
    project_id=project_id,
    if_exists='replace',
    verbose=True,  # This shows progress updates
    progress_bar=True  # Shows a progress bar
)

end_time = time.time()
print(f"Upload completed in {end_time - start_time:.2f} seconds")

Starting upload of 6 rows to BigQuery...
Target: proof-of-brand.social_media_metrics.instagram_data


/usr/local/lib/python3.11/dist-packages/pandas_gbq/gbq.py:528: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00, 1999.19it/s]

Upload completed in 3.34 seconds


### X API Connection

### Define X Bearer Token

In [22]:
import requests
import time

# Replace with your own X Bearer Token
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAABpl1wEAAAAA29Dr%2F5hqGWc0xSZ8uMZqEi3nmdQ%3DuxcTuvO2rqyE4ozsLTInBK6Sx6YIK6MRXFo7mcSd2SJokjIKOk"

### Retrieve tweets by API Connection by tweet id

In [46]:
def get_tweet_metrics(tweet_ids, max_retries=5):
    url = "https://api.twitter.com/2/tweets"

    headers = {
        "Authorization": f"Bearer {BEARER_TOKEN}"
    }

    params = {
        "ids": ",".join(tweet_ids),
        "tweet.fields": "public_metrics"
    }

    retries = 0
    while retries <= max_retries:
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            return response.json()

        elif response.status_code == 429:
            # Too many requests — wait and retry
            wait_time = int(response.headers.get("x-rate-limit-reset", time.time() + 60)) - int(time.time())
            wait_time = max(wait_time, 5)  # Minimum wait time
            print(f"Rate limit hit. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
            retries += 1

        else:
            # Other errors — stop
            raise Exception(f"Request failed: {response.status_code} - {response.text}")

    raise Exception("Max retries exceeded.")

# Example usage
tweet_ids = ['1924504474158006284', '1924505694553641438']  # Up to 100 tweet IDs
metrics = get_tweet_metrics(tweet_ids)

# Display metrics
for tweet in metrics.get("data", []):
    print(f"Tweet ID: {tweet['id']}")
    print(f"  Likes: {tweet['public_metrics']['like_count']}")
    print(f"  Retweets: {tweet['public_metrics']['retweet_count']}")
    print(f"  Replies: {tweet['public_metrics']['reply_count']}")
    print(f"  Quotes: {tweet['public_metrics']['quote_count']}")
    print()


Rate limit hit. Retrying in 615 seconds...
Tweet ID: 1924504474158006284
  Likes: 1
  Retweets: 0
  Replies: 0
  Quotes: 0

Tweet ID: 1924505694553641438
  Likes: 0
  Retweets: 0
  Replies: 0
  Quotes: 0



### Retrieve all tweets from a particular X account

In [47]:
import requests
import time
import pandas as pd
from google.cloud import bigquery
from datetime import datetime
import psutil

# === Step 1: Twitter Bearer Token ===
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAABpl1wEAAAAA29Dr%2F5hqGWc0xSZ8uMZqEi3nmdQ%3DuxcTuvO2rqyE4ozsLTInBK6Sx6YIK6MRXFo7mcSd2SJokjIKOk"

def get_user_id(username):
    print(f"🔍 Looking up user ID for: @{username}")
    start_time = time.time()

    url = f"https://api.twitter.com/2/users/by/username/{username}"
    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}

    response = requests.get(url, headers=headers)
    duration = time.time() - start_time

    if response.status_code == 200:
        user_id = response.json()["data"]["id"]
        print(f"✅ Found user ID: {user_id} (took {duration:.2f}s)")
        return user_id
    else:
        print(f"❌ Failed to get user ID: {response.status_code} - {response.text}")
        raise Exception(f"Failed to get user ID: {response.status_code} - {response.text}")

def get_all_tweets(user_id, max_results=100):
    print(f"\n🐦 Starting tweet collection for user ID: {user_id}")
    print(f"📊 Max results per request: {max_results}")

    url = f"https://api.twitter.com/2/users/{user_id}/tweets"
    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    params = {
        "tweet.fields": "public_metrics,created_at",
        "max_results": max_results
    }

    all_tweets = []
    next_token = None
    request_count = 0
    total_start_time = time.time()

    while True:
        request_count += 1
        print(f"\n📡 API Request #{request_count}")

        if next_token:
            params["pagination_token"] = next_token
            print(f"🔗 Using pagination token: {next_token[:20]}...")

        request_start = time.time()
        response = requests.get(url, headers=headers, params=params)
        request_duration = time.time() - request_start

        print(f"Response: {response.status_code} (took {request_duration:.2f}s)")

        if response.status_code == 200:
            data = response.json()
            batch_tweets = data.get("data", [])
            all_tweets.extend(batch_tweets)

            print(f"📥 Retrieved {len(batch_tweets)} tweets (Total: {len(all_tweets)})")

            # Rate limit info
            remaining = response.headers.get("x-rate-limit-remaining", "Unknown")
            reset_time = response.headers.get("x-rate-limit-reset", "Unknown")
            print(f"🔄 Rate limit: {remaining} requests remaining")

            next_token = data.get("meta", {}).get("next_token")
            if not next_token:
                print("✅ No more tweets to fetch")
                break

            print("⏸️  Waiting 1s before next request...")
            time.sleep(1)

        elif response.status_code == 429:
            wait_time = int(response.headers.get("x-rate-limit-reset", time.time() + 60)) - int(time.time())
            wait_time = max(wait_time, 5)
            print(f"⏳ Rate limit hit. Waiting {wait_time} seconds...")
            print(f"Reset time: {datetime.fromtimestamp(int(response.headers.get('x-rate-limit-reset', time.time())))}")
            time.sleep(wait_time)
        else:
            print(f"❌ API Error: {response.status_code} - {response.text}")
            raise Exception(f"Failed to get tweets: {response.status_code} - {response.text}")

    total_duration = time.time() - total_start_time
    print(f"\n🏁 Tweet collection completed!")
    print(f"📊 Total tweets collected: {len(all_tweets)}")
    print(f"⏱️  Total time: {total_duration:.2f}s")
    print(f"📡 API requests made: {request_count}")
    print(f"📈 Average tweets per request: {len(all_tweets)/request_count:.1f}")

    return all_tweets

def tweets_to_dataframe(tweets):
    print(f"\n📋 Converting {len(tweets)} tweets to DataFrame...")
    start_time = time.time()

    df = pd.DataFrame([
        {
            "tweet_id": tweet["id"],
            "text": tweet["text"],
            "created_at": tweet["created_at"],
            "like_count": tweet["public_metrics"]["like_count"],
            "retweet_count": tweet["public_metrics"]["retweet_count"],
            "reply_count": tweet["public_metrics"]["reply_count"],
            "quote_count": tweet["public_metrics"]["quote_count"],
            "Platform": "X",
        }
        for tweet in tweets
    ])

    duration = time.time() - start_time
    memory_usage = df.memory_usage(deep=True).sum() / 1024**2

    print(f"✅ DataFrame created in {duration:.2f}s")
    print(f"📊 Shape: {df.shape}")
    print(f"💾 Memory usage: {memory_usage:.2f} MB")

    # Show engagement stats
    total_engagement = df[['like_count', 'retweet_count', 'reply_count', 'quote_count']].sum().sum()
    avg_engagement = total_engagement / len(df)
    print(f"📈 Total engagement: {total_engagement:,}")
    print(f"📊 Average engagement per tweet: {avg_engagement:.1f}")

    return df

# === Step 2: Main execution with comprehensive logging ===
print("=" * 60)
print("🚀 TWITTER DATA COLLECTION STARTED")
print("=" * 60)

# Monitor system resources
process = psutil.Process()
memory_start = process.memory_info().rss / 1024**2
print(f"🔍 Initial memory usage: {memory_start:.2f} MB")

overall_start = time.time()

try:
    username = "mnblanalytics"
    print(f"🎯 Target username: @{username}")

    # Step 1: Get user ID
    user_id = get_user_id(username)

    # Step 2: Get all tweets
    tweets = get_all_tweets(user_id)

    # Step 3: Convert to DataFrame
    df_tweets = tweets_to_dataframe(tweets)

    # Final stats
    overall_duration = time.time() - overall_start
    memory_end = process.memory_info().rss / 1024**2
    memory_increase = memory_end - memory_start

    print("\n" + "=" * 60)
    print("🎉 COLLECTION COMPLETED SUCCESSFULLY!")
    print("=" * 60)
    print(f"⏱️  Total execution time: {overall_duration:.2f}s")
    print(f"📊 Final dataset: {df_tweets.shape[0]} tweets, {df_tweets.shape[1]} columns")
    print(f"💾 Memory increase: +{memory_increase:.2f} MB")
    print(f"📈 Processing rate: {len(tweets)/overall_duration:.1f} tweets/second")

    # Preview with engagement metrics
    print(f"\n📋 DATASET PREVIEW:")
    print("-" * 40)
    print(df_tweets.head())

    print(f"\n📊 TOP PERFORMING TWEETS:")
    print("-" * 40)
    df_tweets['total_engagement'] = df_tweets[['like_count', 'retweet_count', 'reply_count', 'quote_count']].sum(axis=1)
    top_tweets = df_tweets.nlargest(3, 'total_engagement')[['tweet_id', 'text', 'total_engagement']]
    for _, tweet in top_tweets.iterrows():
        print(f"ID: {tweet['tweet_id']} | Engagement: {tweet['total_engagement']:,}")
        print(f"Text: {tweet['text'][:100]}...")
        print()

except Exception as e:
    overall_duration = time.time() - overall_start
    memory_end = process.memory_info().rss / 1024**2

    print(f"\n❌ EXECUTION FAILED after {overall_duration:.2f}s")
    print(f"💾 Memory at failure: {memory_end:.2f} MB")
    print(f"Error: {str(e)}")
    raise

🚀 TWITTER DATA COLLECTION STARTED
🔍 Initial memory usage: 1878.62 MB
🎯 Target username: @mnblanalytics
🔍 Looking up user ID for: @mnblanalytics
✅ Found user ID: 1924492647856578560 (took 0.10s)

🐦 Starting tweet collection for user ID: 1924492647856578560
📊 Max results per request: 100

📡 API Request #1
Response: 200 (took 0.10s)
📥 Retrieved 2 tweets (Total: 2)
🔄 Rate limit: 0 requests remaining
✅ No more tweets to fetch

🏁 Tweet collection completed!
📊 Total tweets collected: 2
⏱️  Total time: 0.10s
📡 API requests made: 1
📈 Average tweets per request: 2.0

📋 Converting 2 tweets to DataFrame...
✅ DataFrame created in 0.00s
📊 Shape: (2, 8)
💾 Memory usage: 0.00 MB
📈 Total engagement: 1
📊 Average engagement per tweet: 0.5

🎉 COLLECTION COMPLETED SUCCESSFULLY!
⏱️  Total execution time: 0.21s
📊 Final dataset: 2 tweets, 8 columns
💾 Memory increase: +0.00 MB
📈 Processing rate: 9.7 tweets/second

📋 DATASET PREVIEW:
----------------------------------------
              tweet_id                

In [ ]:
#Add permalink, platform, username, followers, media_type

### Generate X DDBB into Big Query

In [48]:
from google.cloud import bigquery
from pandas_gbq import to_gbq
import time
import psutil
from datetime import datetime

project_id = "proof-of-brand"
dataset_id = "social_media_metrics"
table_id_tweets = "tweets_data"

print("=" * 60)
print("📤 BIGQUERY UPLOAD STARTED")
print("=" * 60)

# Monitor system resources
process = psutil.Process()
memory_before = process.memory_info().rss / 1024**2
print(f"🔍 Memory before upload: {memory_before:.2f} MB")

# Analyze DataFrame before upload
print(f"\n📊 DATASET ANALYSIS:")
print("-" * 30)
print(f"Shape: {df_tweets.shape[0]} rows × {df_tweets.shape[1]} columns")
print(f"Memory usage: {df_tweets.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Target table: {project_id}.{dataset_id}.{table_id_tweets}")

# Show data types and potential issues
print(f"\n📋 DATA TYPES:")
for col, dtype in df_tweets.dtypes.items():
    null_count = df_tweets[col].isnull().sum()
    print(f"  {col}: {dtype} ({null_count} nulls)")

# Check for any data quality issues
print(f"\n🔍 DATA QUALITY CHECK:")
duplicate_rows = df_tweets.duplicated().sum()
print(f"  Duplicate rows: {duplicate_rows}")
print(f"  Date range: {df_tweets['created_at'].min()} to {df_tweets['created_at'].max()}")

# Start upload with timing
print(f"\n🚀 Starting upload at {datetime.now().strftime('%H:%M:%S')}")
upload_start = time.time()

try:
    # Upload with verbose output
    to_gbq(
        df_tweets,
        f"{dataset_id}.{table_id_tweets}",
        project_id=project_id,
        if_exists='replace',
        verbose=True,  # Shows progress
        progress_bar=True  # Shows progress bar
    )

    upload_duration = time.time() - upload_start
    memory_after = process.memory_info().rss / 1024**2
    memory_change = memory_after - memory_before

    # Success metrics
    print(f"\n✅ UPLOAD SUCCESSFUL!")
    print("=" * 40)
    print(f"⏱️  Upload time: {upload_duration:.2f} seconds")
    print(f"📊 Rows uploaded: {df_tweets.shape[0]:,}")
    print(f"📈 Upload rate: {df_tweets.shape[0]/upload_duration:.1f} rows/second")
    print(f"💾 Memory change: {memory_change:+.2f} MB")
    print(f"📦 Data size: {df_tweets.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    # Calculate some engagement stats for the upload
    total_engagement = df_tweets[['like_count', 'retweet_count', 'reply_count', 'quote_count']].sum().sum()
    print(f"📈 Total engagement uploaded: {total_engagement:,}")

except Exception as e:
    upload_duration = time.time() - upload_start
    memory_after = process.memory_info().rss / 1024**2

    print(f"\n❌ UPLOAD FAILED after {upload_duration:.2f}s")
    print(f"💾 Memory at failure: {memory_after:.2f} MB")
    print(f"Error: {str(e)}")
    raise

print(f"\n🎯 Table location: https://console.cloud.google.com/bigquery?project={project_id}")

📤 BIGQUERY UPLOAD STARTED
🔍 Memory before upload: 1878.62 MB

📊 DATASET ANALYSIS:
------------------------------
Shape: 2 rows × 9 columns
Memory usage: 0.00 MB
Target table: proof-of-brand.social_media_metrics.tweets_data

📋 DATA TYPES:
  tweet_id: object (0 nulls)
  text: object (0 nulls)
  created_at: object (0 nulls)
  like_count: int64 (0 nulls)
  retweet_count: int64 (0 nulls)
  reply_count: int64 (0 nulls)
  quote_count: int64 (0 nulls)
  Platform: object (0 nulls)
  total_engagement: int64 (0 nulls)

🔍 DATA QUALITY CHECK:
  Duplicate rows: 0
  Date range: 2025-05-19T16:36:36.000Z to 2025-05-19T16:41:27.000Z

🚀 Starting upload at 15:16:38


/usr/local/lib/python3.11/dist-packages/pandas_gbq/gbq.py:528: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00, 2045.00it/s]


✅ UPLOAD SUCCESSFUL!
⏱️  Upload time: 3.60 seconds
📊 Rows uploaded: 2
📈 Upload rate: 0.6 rows/second
💾 Memory change: +0.00 MB
📦 Data size: 0.00 MB
📈 Total engagement uploaded: 1

🎯 Table location: https://console.cloud.google.com/bigquery?project=proof-of-brand


### X Sentiment Analysis

In [49]:
import tweepy
import re
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import time
import psutil
from datetime import datetime

print("=" * 60)
print("🧠 SENTIMENT ANALYSIS PIPELINE STARTED")
print("=" * 60)

# Monitor system resources
process = psutil.Process()
memory_start = process.memory_info().rss / 1024**2
print(f"🔍 Initial memory usage: {memory_start:.2f} MB")

overall_start = time.time()

# ------------------------------
# 1. SETUP: Twitter API
# ------------------------------
print(f"\n🔧 SETTING UP TWITTER API")
print("-" * 30)

bearer_token = BEARER_TOKEN
print(f"🔑 Bearer token configured: {bearer_token[:20]}...")

api_start = time.time()
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)
api_duration = time.time() - api_start
print(f"✅ API client initialized in {api_duration:.2f}s")

# ------------------------------
# 2. QUERY & FETCH TWEETS
# ------------------------------
print(f"\n🔍 FETCHING TWEETS")
print("-" * 30)

query = '("Fintual" OR "#Fintual") lang:en -is:retweet'
max_results = 10

print(f"🎯 Query: {query}")
print(f"📊 Max results: {max_results}")

fetch_start = time.time()

try:
    print(f"🚀 Starting tweet search at {datetime.now().strftime('%H:%M:%S')}")
    response = client.search_recent_tweets(query=query, max_results=max_results, tweet_fields=['text'])
    fetch_duration = time.time() - fetch_start

    tweets = [tweet.text for tweet in response.data] if response.data else []

    print(f"✅ Search completed in {fetch_duration:.2f}s")
    print(f"📥 Retrieved {len(tweets)} tweets")

    if tweets:
        avg_length = sum(len(tweet) for tweet in tweets) / len(tweets)
        print(f"📝 Average tweet length: {avg_length:.1f} characters")

except tweepy.errors.TooManyRequests as e:
    fetch_duration = time.time() - fetch_start
    print(f"⏳ Rate limit hit after {fetch_duration:.2f}s: {e}")
    print("🔄 Waiting 60s before retry...")
    time.sleep(60)

    retry_start = time.time()
    response = client.search_recent_tweets(query=query, max_results=max_results, tweet_fields=['text'])
    retry_duration = time.time() - retry_start

    tweets = [tweet.text for tweet in response.data] if response.data else []
    print(f"✅ Retry successful in {retry_duration:.2f}s")
    print(f"📥 Retrieved {len(tweets)} tweets")

# ------------------------------
# 3. CLEANING FUNCTION
# ------------------------------
print(f"\n🧹 TWEET CLEANING")
print("-" * 30)

def clean_tweet(tweet):
    original_length = len(tweet)
    tweet = re.sub(r"http\S+", "", tweet)  # remove URLs
    tweet = re.sub(r"@\w+", "", tweet)     # remove mentions
    tweet = re.sub(r"#", "", tweet)        # remove hashtag symbol
    tweet = re.sub(r"\s+", " ", tweet).strip()
    cleaned_length = len(tweet)
    return tweet, original_length, cleaned_length

# Clean all tweets and track statistics
cleaning_start = time.time()
cleaning_stats = []

for i, tweet in enumerate(tweets):
    clean_text, orig_len, clean_len = clean_tweet(tweet)
    tweets[i] = clean_text  # Update the original list
    cleaning_stats.append({
        'original_length': orig_len,
        'cleaned_length': clean_len,
        'reduction': orig_len - clean_len
    })

cleaning_duration = time.time() - cleaning_start

if cleaning_stats:
    avg_reduction = sum(stat['reduction'] for stat in cleaning_stats) / len(cleaning_stats)
    print(f"✅ Cleaned {len(tweets)} tweets in {cleaning_duration:.3f}s")
    print(f"📊 Average character reduction: {avg_reduction:.1f}")

# ------------------------------
# 4. LOAD SENTIMENT MODEL
# ------------------------------
print(f"\n🤖 LOADING SENTIMENT MODEL")
print("-" * 30)

MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
print(f"📦 Model: {MODEL}")

model_start = time.time()

print("🔄 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL)

print("🔄 Loading model...")
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

model_duration = time.time() - model_start
memory_after_model = process.memory_info().rss / 1024**2
model_memory = memory_after_model - memory_start

labels = ['Negative', 'Neutral', 'Positive']

print(f"✅ Model loaded in {model_duration:.2f}s")
print(f"💾 Model memory usage: +{model_memory:.2f} MB")
print(f"🏷️  Labels: {labels}")

# ------------------------------
# 5. PREDICT SENTIMENTS
# ------------------------------
print(f"\n🎭 SENTIMENT ANALYSIS")
print("-" * 30)

def get_sentiment(text):
    start_time = time.time()
    encoded_input = tokenizer(text, return_tensors='pt', truncation=True)

    with torch.no_grad():
        output = model(**encoded_input)

    scores = output.logits[0].numpy()
    scores = softmax(scores)
    sentiment = labels[scores.argmax()]
    confidence = scores.max()
    duration = time.time() - start_time

    return sentiment, float(confidence), duration

# ------------------------------
# 6. ANALYZE AND OUTPUT RESULTS
# ------------------------------
print(f"🚀 Starting analysis of {len(tweets)} tweets...")

analysis_start = time.time()
results = []
prediction_times = []

for i, tweet in enumerate(tweets, 1):
    print(f"  Processing tweet {i}/{len(tweets)}...", end=" ")

    sentiment, confidence, pred_time = get_sentiment(tweet)
    prediction_times.append(pred_time)

    results.append({
        'tweet': tweet,
        'sentiment': sentiment,
        'confidence': round(confidence, 3)
    })

    print(f"{sentiment} ({confidence:.3f}) - {pred_time:.3f}s")

analysis_duration = time.time() - analysis_start
overall_duration = time.time() - overall_start
memory_final = process.memory_info().rss / 1024**2

# Create DataFrame and show results
df = pd.DataFrame(results)

print(f"\n📊 ANALYSIS COMPLETE!")
print("=" * 40)
print(f"⏱️  Total execution time: {overall_duration:.2f}s")
print(f"🎭 Analysis time: {analysis_duration:.2f}s")
print(f"📈 Average prediction time: {sum(prediction_times)/len(prediction_times):.3f}s")
print(f"💾 Total memory usage: {memory_final:.2f} MB (+{memory_final - memory_start:.2f} MB)")

# Sentiment distribution
if not df.empty:
    sentiment_counts = df['sentiment'].value_counts()
    print(f"\n📊 SENTIMENT DISTRIBUTION:")
    print("-" * 30)
    for sentiment, count in sentiment_counts.items():
        percentage = (count / len(df)) * 100
        print(f"  {sentiment}: {count} tweets ({percentage:.1f}%)")

    avg_confidence = df['confidence'].mean()
    print(f"\n🎯 Average confidence: {avg_confidence:.3f}")

    print(f"\n📋 RESULTS PREVIEW:")
    print("-" * 40)
    print(df.head(10))

    # Show most confident predictions
    print(f"\n🏆 MOST CONFIDENT PREDICTIONS:")
    print("-" * 40)
    top_confident = df.nlargest(3, 'confidence')
    for _, row in top_confident.iterrows():
        print(f"Sentiment: {row['sentiment']} (confidence: {row['confidence']})")
        print(f"Tweet: {row['tweet'][:100]}...")
        print()

else:
    print("⚠️  No tweets to analyze")

print(f"\n✅ Pipeline completed successfully!")

🧠 SENTIMENT ANALYSIS PIPELINE STARTED
🔍 Initial memory usage: 1878.62 MB

🔧 SETTING UP TWITTER API
------------------------------
🔑 Bearer token configured: AAAAAAAAAAAAAAAAAAAA...
✅ API client initialized in 0.00s

🔍 FETCHING TWEETS
------------------------------
🎯 Query: ("Fintual" OR "#Fintual") lang:en -is:retweet
📊 Max results: 10
🚀 Starting tweet search at 15:17:53
✅ Search completed in 0.13s
📥 Retrieved 1 tweets
📝 Average tweet length: 228.0 characters

🧹 TWEET CLEANING
------------------------------
✅ Cleaned 1 tweets in 0.000s
📊 Average character reduction: 30.0

🤖 LOADING SENTIMENT MODEL
------------------------------
📦 Model: cardiffnlp/twitter-roberta-base-sentiment
🔄 Loading tokenizer...
🔄 Loading model...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Model loaded in 1.44s
💾 Model memory usage: +204.73 MB
🏷️  Labels: ['Negative', 'Neutral', 'Positive']

🎭 SENTIMENT ANALYSIS
------------------------------
🚀 Starting analysis of 1 tweets...
  Processing tweet 1/1... 

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Positive (0.990) - 0.305s

📊 ANALYSIS COMPLETE!
⏱️  Total execution time: 1.88s
🎭 Analysis time: 0.31s
📈 Average prediction time: 0.305s
💾 Total memory usage: 2074.42 MB (+195.79 MB)

📊 SENTIMENT DISTRIBUTION:
------------------------------
  Positive: 1 tweets (100.0%)

🎯 Average confidence: 0.990

📋 RESULTS PREVIEW:
----------------------------------------
                                               tweet sentiment  confidence
0  160 confirmed attendees, 500+ registrations. I...  Positive        0.99

🏆 MOST CONFIDENT PREDICTIONS:
----------------------------------------
Sentiment: Positive (confidence: 0.99)
Tweet: 160 confirmed attendees, 500+ registrations. I'm excited to launch our AI community in CDMX. Gratefu...


✅ Pipeline completed successfully!


In [50]:
print(tweet)

160 confirmed attendees, 500+ registrations. I'm excited to launch our AI community in CDMX. Grateful to cursor_ai an stripe for sponsoring our event and to fintual for partnering. See you Thursday!


### Obtain last tweets associated to specific theme

In [51]:
import tweepy
import re
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import time
import psutil
from datetime import datetime

print("=" * 60)
print("🥤 COCA-COLA SENTIMENT ANALYSIS PIPELINE")
print("=" * 60)

# Monitor system resources
process = psutil.Process()
memory_start = process.memory_info().rss / 1024**2
print(f"🔍 Initial memory usage: {memory_start:.2f} MB")
print(f"🕐 Started at: {datetime.now().strftime('%H:%M:%S')}")

overall_start = time.time()

# ------------------------------
# 1. SETUP: Twitter API
# ------------------------------
print(f"\n🔧 SETTING UP TWITTER API")
print("-" * 30)

bearer_token = BEARER_TOKEN
print(f"🔑 Bearer token configured: {bearer_token[:20]}...")

api_start = time.time()
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)
api_duration = time.time() - api_start
print(f"✅ API client initialized in {api_duration:.2f}s")

# ------------------------------
# 2. QUERY & FETCH TWEETS
# ------------------------------
print(f"\n🔍 FETCHING COCA-COLA TWEETS")
print("-" * 30)

query = '("Coca-cola" OR "#cocacola") lang:en -is:retweet'
max_results = 10

print(f"🎯 Query: {query}")
print(f"📊 Max results: {max_results}")
print(f"🌐 Language: English only")
print(f"🚫 Excluding retweets")

fetch_start = time.time()

try:
    print(f"🚀 Starting tweet search at {datetime.now().strftime('%H:%M:%S')}")
    response = client.search_recent_tweets(query=query, max_results=max_results, tweet_fields=['text'])
    fetch_duration = time.time() - fetch_start

    tweets = [tweet.text for tweet in response.data] if response.data else []

    print(f"✅ Search completed in {fetch_duration:.2f}s")
    print(f"📥 Retrieved {len(tweets)} tweets")

    if tweets:
        avg_length = sum(len(tweet) for tweet in tweets) / len(tweets)
        max_length = max(len(tweet) for tweet in tweets)
        min_length = min(len(tweet) for tweet in tweets)
        print(f"📝 Tweet length stats:")
        print(f"   Average: {avg_length:.1f} characters")
        print(f"   Range: {min_length}-{max_length} characters")
    else:
        print("⚠️  No tweets found for this query")

except tweepy.errors.TooManyRequests as e:
    fetch_duration = time.time() - fetch_start
    print(f"⏳ Rate limit hit after {fetch_duration:.2f}s: {e}")
    print("🔄 Waiting 60s before retry...")

    wait_start = time.time()
    time.sleep(60)
    wait_duration = time.time() - wait_start

    print(f"⏰ Waited {wait_duration:.1f}s, retrying...")
    retry_start = time.time()
    response = client.search_recent_tweets(query=query, max_results=max_results, tweet_fields=['text'])
    retry_duration = time.time() - retry_start

    tweets = [tweet.text for tweet in response.data] if response.data else []
    print(f"✅ Retry successful in {retry_duration:.2f}s")
    print(f"📥 Retrieved {len(tweets)} tweets")

# ------------------------------
# 3. CLEANING FUNCTION
# ------------------------------
print(f"\n🧹 TWEET CLEANING")
print("-" * 30)

def clean_tweet(tweet):
    original_length = len(tweet)

    # Count elements being removed
    urls = len(re.findall(r"http\S+", tweet))
    mentions = len(re.findall(r"@\w+", tweet))
    hashtags = len(re.findall(r"#\w+", tweet))

    tweet = re.sub(r"http\S+", "", tweet)  # remove URLs
    tweet = re.sub(r"@\w+", "", tweet)     # remove mentions
    tweet = re.sub(r"#", "", tweet)        # remove hashtag symbol
    tweet = re.sub(r"\s+", " ", tweet).strip()

    cleaned_length = len(tweet)

    return tweet, {
        'original_length': original_length,
        'cleaned_length': cleaned_length,
        'urls_removed': urls,
        'mentions_removed': mentions,
        'hashtags_removed': hashtags,
        'reduction': original_length - cleaned_length
    }

# Clean all tweets and track statistics
cleaning_start = time.time()
cleaning_stats = []
cleaned_tweets = []

print(f"🔄 Processing {len(tweets)} tweets...")

for i, tweet in enumerate(tweets, 1):
    clean_text, stats = clean_tweet(tweet)
    cleaned_tweets.append(clean_text)
    cleaning_stats.append(stats)

    print(f"   Tweet {i}: {stats['original_length']}→{stats['cleaned_length']} chars "
          f"(URLs:{stats['urls_removed']}, @:{stats['mentions_removed']}, #:{stats['hashtags_removed']})")

cleaning_duration = time.time() - cleaning_start

if cleaning_stats:
    total_reduction = sum(stat['reduction'] for stat in cleaning_stats)
    avg_reduction = total_reduction / len(cleaning_stats)
    total_urls = sum(stat['urls_removed'] for stat in cleaning_stats)
    total_mentions = sum(stat['mentions_removed'] for stat in cleaning_stats)
    total_hashtags = sum(stat['hashtags_removed'] for stat in cleaning_stats)

    print(f"\n✅ Cleaning completed in {cleaning_duration:.3f}s")
    print(f"📊 Cleaning summary:")
    print(f"   Total character reduction: {total_reduction}")
    print(f"   Average reduction per tweet: {avg_reduction:.1f}")
    print(f"   Elements removed: {total_urls} URLs, {total_mentions} mentions, {total_hashtags} hashtags")

# ------------------------------
# 4. LOAD SENTIMENT MODEL
# ------------------------------
print(f"\n🤖 LOADING SENTIMENT MODEL")
print("-" * 30)

MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
print(f"📦 Model: {MODEL}")

model_start = time.time()

print("🔄 Loading tokenizer...")
tokenizer_start = time.time()
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer_duration = time.time() - tokenizer_start

print("🔄 Loading model...")
model_load_start = time.time()
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model_load_duration = time.time() - model_load_start

model_duration = time.time() - model_start
memory_after_model = process.memory_info().rss / 1024**2
model_memory = memory_after_model - memory_start

labels = ['Negative', 'Neutral', 'Positive']

print(f"✅ Model components loaded:")
print(f"   Tokenizer: {tokenizer_duration:.2f}s")
print(f"   Model: {model_load_duration:.2f}s")
print(f"   Total: {model_duration:.2f}s")
print(f"💾 Model memory usage: +{model_memory:.2f} MB")
print(f"🏷️  Sentiment labels: {labels}")

# ------------------------------
# 5. PREDICT SENTIMENTS
# ------------------------------
print(f"\n🎭 SENTIMENT ANALYSIS")
print("-" * 30)

def get_sentiment(text):
    start_time = time.time()

    # Tokenization timing
    tokenize_start = time.time()
    encoded_input = tokenizer(text, return_tensors='pt', truncation=True)
    tokenize_duration = time.time() - tokenize_start

    # Model inference timing
    inference_start = time.time()
    with torch.no_grad():
        output = model(**encoded_input)
    inference_duration = time.time() - inference_start

    # Post-processing timing
    postprocess_start = time.time()
    scores = output.logits[0].numpy()
    scores = softmax(scores)
    sentiment = labels[scores.argmax()]
    confidence = scores.max()
    postprocess_duration = time.time() - postprocess_start

    total_duration = time.time() - start_time

    return sentiment, float(confidence), {
        'total_time': total_duration,
        'tokenize_time': tokenize_duration,
        'inference_time': inference_duration,
        'postprocess_time': postprocess_duration
    }

# ------------------------------
# 6. ANALYZE AND OUTPUT RESULTS
# ------------------------------
print(f"🚀 Starting sentiment analysis of {len(cleaned_tweets)} tweets...")

analysis_start = time.time()
results = []
timing_stats = []

for i, tweet in enumerate(cleaned_tweets, 1):
    print(f"  🔄 Processing tweet {i}/{len(cleaned_tweets)}...", end=" ")

    sentiment, confidence, timing = get_sentiment(tweet)
    timing_stats.append(timing)

    results.append({
        'tweet': tweet,
        'sentiment': sentiment,
        'confidence': round(confidence, 3),
        'Platform': 'X'
    })

    print(f"{sentiment} ({confidence:.3f}) - {timing['total_time']:.3f}s")

analysis_duration = time.time() - analysis_start
overall_duration = time.time() - overall_start
memory_final = process.memory_info().rss / 1024**2

# Create DataFrame
df_tweets_sentiment = pd.DataFrame(results)

print(f"\n🎉 ANALYSIS COMPLETE!")
print("=" * 50)
print(f"⏱️  Total pipeline time: {overall_duration:.2f}s")
print(f"🎭 Analysis phase time: {analysis_duration:.2f}s")
print(f"💾 Final memory usage: {memory_final:.2f} MB (+{memory_final - memory_start:.2f} MB)")

# Timing breakdown
if timing_stats:
    avg_total = sum(t['total_time'] for t in timing_stats) / len(timing_stats)
    avg_tokenize = sum(t['tokenize_time'] for t in timing_stats) / len(timing_stats)
    avg_inference = sum(t['inference_time'] for t in timing_stats) / len(timing_stats)
    avg_postprocess = sum(t['postprocess_time'] for t in timing_stats) / len(timing_stats)

    print(f"\n⚡ PERFORMANCE BREAKDOWN:")
    print(f"   Average per tweet: {avg_total:.3f}s")
    print(f"   Tokenization: {avg_tokenize:.3f}s ({avg_tokenize/avg_total*100:.1f}%)")
    print(f"   Inference: {avg_inference:.3f}s ({avg_inference/avg_total*100:.1f}%)")
    print(f"   Post-processing: {avg_postprocess:.3f}s ({avg_postprocess/avg_total*100:.1f}%)")

# Sentiment analysis results
if not df_tweets_sentiment.empty:
    sentiment_counts = df_tweets_sentiment['sentiment'].value_counts()
    print(f"\n🥤 COCA-COLA SENTIMENT DISTRIBUTION:")
    print("-" * 40)
    for sentiment, count in sentiment_counts.items():
        percentage = (count / len(df_tweets_sentiment)) * 100
        emoji = "😊" if sentiment == "Positive" else "😐" if sentiment == "Neutral" else "😞"
        print(f"   {emoji} {sentiment}: {count} tweets ({percentage:.1f}%)")

    avg_confidence = df_tweets_sentiment['confidence'].mean()
    high_confidence = (df_tweets_sentiment['confidence'] >= 0.8).sum()

    print(f"\n🎯 CONFIDENCE METRICS:")
    print(f"   Average confidence: {avg_confidence:.3f}")
    print(f"   High confidence (≥0.8): {high_confidence}/{len(df_tweets_sentiment)} tweets")

    print(f"\n📋 RESULTS PREVIEW:")
    print("-" * 50)
    print(df_tweets_sentiment.head(10))

    # Show most confident predictions by sentiment
    print(f"\n🏆 MOST CONFIDENT PREDICTIONS:")
    print("-" * 40)
    for sentiment in ['Positive', 'Negative', 'Neutral']:
        sentiment_tweets = df_tweets_sentiment[df_tweets_sentiment['sentiment'] == sentiment]
        if not sentiment_tweets.empty:
            top_tweet = sentiment_tweets.loc[sentiment_tweets['confidence'].idxmax()]
            emoji = "😊" if sentiment == "Positive" else "😐" if sentiment == "Neutral" else "😞"
            print(f"{emoji} {sentiment} (confidence: {top_tweet['confidence']}):")
            print(f"   \"{top_tweet['tweet'][:80]}...\"")
            print()

else:
    print("⚠️  No tweets to analyze")

print(f"✅ Coca-Cola sentiment analysis pipeline completed!")
print(f"🕐 Finished at: {datetime.now().strftime('%H:%M:%S')}")

🥤 COCA-COLA SENTIMENT ANALYSIS PIPELINE
🔍 Initial memory usage: 2074.42 MB
🕐 Started at: 15:19:50

🔧 SETTING UP TWITTER API
------------------------------
🔑 Bearer token configured: AAAAAAAAAAAAAAAAAAAA...
✅ API client initialized in 0.00s

🔍 FETCHING COCA-COLA TWEETS
------------------------------
🎯 Query: ("Coca-cola" OR "#cocacola") lang:en -is:retweet
📊 Max results: 10
🌐 Language: English only
🚫 Excluding retweets
🚀 Starting tweet search at 15:19:50
✅ Search completed in 784.22s
📥 Retrieved 10 tweets
📝 Tweet length stats:
   Average: 239.8 characters
   Range: 72-301 characters

🧹 TWEET CLEANING
------------------------------
🔄 Processing 10 tweets...
   Tweet 1: 260→260 chars (URLs:0, @:0, #:0)
   Tweet 2: 200→190 chars (URLs:0, @:1, #:0)
   Tweet 3: 245→245 chars (URLs:0, @:0, #:0)
   Tweet 4: 290→266 chars (URLs:1, @:0, #:0)
   Tweet 5: 72→55 chars (URLs:0, @:1, #:0)
   Tweet 6: 301→265 chars (URLs:1, @:1, #:0)
   Tweet 7: 254→240 chars (URLs:0, @:1, #:0)
   Tweet 8: 249→240 cha

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Model components loaded:
   Tokenizer: 0.27s
   Model: 0.33s
   Total: 0.60s
💾 Model memory usage: +3.13 MB
🏷️  Sentiment labels: ['Negative', 'Neutral', 'Positive']

🎭 SENTIMENT ANALYSIS
------------------------------
🚀 Starting sentiment analysis of 10 tweets...
  🔄 Processing tweet 1/10... 

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Negative (0.807) - 0.282s
  🔄 Processing tweet 2/10... Neutral (0.537) - 0.182s
  🔄 Processing tweet 3/10... Positive (0.911) - 0.285s
  🔄 Processing tweet 4/10... Negative (0.464) - 0.232s
  🔄 Processing tweet 5/10... Neutral (0.512) - 0.089s
  🔄 Processing tweet 6/10... Positive (0.972) - 0.257s
  🔄 Processing tweet 7/10... Negative (0.713) - 0.242s
  🔄 Processing tweet 8/10... Negative (0.713) - 0.230s
  🔄 Processing tweet 9/10... Negative (0.713) - 0.225s
  🔄 Processing tweet 10/10... Negative (0.684) - 0.219s

🎉 ANALYSIS COMPLETE!
⏱️  Total pipeline time: 787.07s
🎭 Analysis phase time: 2.24s
💾 Final memory usage: 1881.23 MB (+-193.19 MB)

⚡ PERFORMANCE BREAKDOWN:
   Average per tweet: 0.224s
   Tokenization: 0.001s (0.4%)
   Inference: 0.223s (99.6%)
   Post-processing: 0.000s (0.1%)

🥤 COCA-COLA SENTIMENT DISTRIBUTION:
----------------------------------------
   😞 Negative: 6 tweets (60.0%)
   😐 Neutral: 2 tweets (20.0%)
   😊 Positive: 2 tweets (20.0%)

🎯 CONFIDENCE METRICS:
   A

### Upload Sentiment DDBB too Big Query

In [52]:
import time
from datetime import datetime

project_id = "proof-of-brand"
dataset_id = "social_media_metrics"
table_id_tweets_sentiment = "tweets_sentiment_data"

print("📤 BIGQUERY UPLOAD - SENTIMENT DATA")
print("=" * 40)

# Pre-upload analysis
print(f"📊 Dataset info:")
print(f"   Rows: {len(df_tweets_sentiment)}")
print(f"   Columns: {df_tweets_sentiment.shape[1]}")
print(f"   Size: {df_tweets_sentiment.memory_usage(deep=True).sum() / 1024:.1f} KB")
print(f"   Target: {project_id}.{dataset_id}.{table_id_tweets_sentiment}")

# Show sentiment distribution being uploaded
if not df_tweets_sentiment.empty:
    sentiment_counts = df_tweets_sentiment['sentiment'].value_counts()
    print(f"   Sentiment breakdown: {dict(sentiment_counts)}")

print(f"\n🚀 Starting upload at {datetime.now().strftime('%H:%M:%S')}")
upload_start = time.time()

try:
    to_gbq(
        df_tweets_sentiment,
        f"{dataset_id}.{table_id_tweets_sentiment}",
        project_id=project_id,
        if_exists='replace',
        verbose=True
    )

    upload_duration = time.time() - upload_start

    print(f"\n✅ UPLOAD SUCCESSFUL!")
    print(f"⏱️  Upload time: {upload_duration:.2f}s")
    print(f"📈 Upload rate: {len(df_tweets_sentiment)/upload_duration:.1f} rows/second")
    print(f"🎯 Table: {project_id}.{dataset_id}.{table_id_tweets_sentiment}")

except Exception as e:
    upload_duration = time.time() - upload_start
    print(f"\n❌ UPLOAD FAILED after {upload_duration:.2f}s")
    print(f"Error: {str(e)}")
    raise

print(f"🔗 View in BigQuery: https://console.cloud.google.com/bigquery?project={project_id}")

📤 BIGQUERY UPLOAD - SENTIMENT DATA
📊 Dataset info:
   Rows: 10
   Columns: 4
   Size: 7.6 KB
   Target: proof-of-brand.social_media_metrics.tweets_sentiment_data
   Sentiment breakdown: {'Negative': np.int64(6), 'Neutral': np.int64(2), 'Positive': np.int64(2)}

🚀 Starting upload at 15:32:57


/usr/local/lib/python3.11/dist-packages/pandas_gbq/gbq.py:528: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00, 11214.72it/s]


✅ UPLOAD SUCCESSFUL!
⏱️  Upload time: 3.61s
📈 Upload rate: 2.8 rows/second
🎯 Table: proof-of-brand.social_media_metrics.tweets_sentiment_data
🔗 View in BigQuery: https://console.cloud.google.com/bigquery?project=proof-of-brand
